### Schema for Downloading USD Rates and Stock Market Securities

The objective of this module is to develop a web scraping pipeline to retrieve the USD exchange rate and specific Colombian state-owned stocks (as well as sovereign debt). This data will serve as the foundation for rigorous statistical analysis and variance modeling

In [69]:
###################
#### Libraries ####
###################


from dataclasses import dataclass
from datetime import datetime
from typing import Optional
import requests
from bs4 import BeautifulSoup




In [56]:
###########
## Dolar ##
###########


url  =  'https://www.google.com/finance/quote/USD-COP?sa=X&ved=2ahUKEwiYzO64nuORAxXITDABHZsDKnYQmY0JegQIDRAv' 
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [57]:
# TRM Dolar
dolar_value = soup.find('div', 'YMlKec fxKbKc').text.strip()
dolar_value = dolar_value.replace('$', '').replace(',', '')
dolar_value = float(dolar_value)
dolar_value

3729.5

In [58]:
### Variation Dolar 
variation = soup.find('div','JwB6zf').text.strip()
variation 


'-1.73%'

In [59]:
variation_level = variation.split(' ')[0]
variation_level = variation_level.replace('+', '').replace('%', '')
variation_level = float(variation_level)
variation_level

-1.73

In [ ]:
dolar_value * variation_level / 100

-64.52035

### Refactoring Code - Dollar Case -

In [ ]:

@dataclass
class DollarQuote:
    price: float
    variation_pct: float
    variation_value: float
    timestamp: str
    download_time: str
    source: str = "Google Finance"
    
    @property
    def is_positive(self) -> bool:
        return self.variation_pct >= 0
    
    @property
    def formatted_price(self) -> str:
        return f"${self.price:,.2f} COP"
    
    @property
    def formatted_variation(self) -> str:
        sign = "+" if self.is_positive else ""
        return f"{sign}{self.variation_pct:.2f}%"
    
    def to_dict(self) -> dict:
        return {
            "price": self.price,
            "variation_pct": self.variation_pct,
            "timestamp": self.timestamp,

            "source": self.source,
            "is_positive": self.is_positive,
        }


class DollarScraperError(Exception):
    pass


class DollarScraper:
    
    URL = "https://www.google.com/finance/quote/USD-COP"
    
    HEADERS = {
        "User-Agent": (
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0.0.0 Safari/537.36"
        )
    }
    
    def __init__(self, timeout: int = 10):
        self.timeout = timeout
        self._download_time: Optional[str] = None
    
    def _fetch_page(self) -> BeautifulSoup:
        try:
            self._download_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            
            response = requests.get(
                self.URL,
                headers=self.HEADERS,
                timeout=self.timeout
            )
            response.raise_for_status()
            return BeautifulSoup(response.content, "html.parser")
        
        except requests.RequestException as e:
            raise DollarScraperError(f"Error al obtener la pagina: {e}")
    
    def _parse_price(self, soup: BeautifulSoup) -> float:
        element = soup.find("div", class_="YMlKec fxKbKc")
        
        if element is None:
            raise DollarScraperError("No se encontro el elemento del precio")
        
        price_text = element.text.strip()
        price_clean = price_text.replace("$", "").replace(",", "")
        
        try:
            return float(price_clean)
        except ValueError:
            raise DollarScraperError(f"No se pudo convertir el precio: {price_text}")
    
    def _parse_variation(self, soup: BeautifulSoup) -> tuple[float, float]:
        element = soup.find("div", class_="JwB6zf")
        
        if element is None:
            raise DollarScraperError("No se encontro el elemento de variacion")
        
        variation_text = element.text.strip()
        
        try:
            # Formato 1: "-0.55%" o "+0.35%"
            if variation_text.endswith("%") and " " not in variation_text:
                pct_text = variation_text.replace("+", "").replace("%", "")
                variation_pct = float(pct_text)
                variation_value = 0.0  # No disponible en este formato
                return variation_pct, variation_value
            
            # Formato 2: "+15.00 (0.35%)" o "-18.50 (0.45%)"
            parts = variation_text.split(" ")
            
            value_text = parts[0].replace("+", "").replace(",", "")
            variation_value = float(value_text)
            
            pct_text = parts[1].replace("(", "").replace(")", "").replace("%", "")
            variation_pct = float(pct_text)
            
            if variation_text.startswith("-"):
                variation_value = -abs(variation_value)
                variation_pct = -abs(variation_pct)
            
            return variation_pct, variation_value
            
        except (IndexError, ValueError) as e:
            raise DollarScraperError(f"No se pudo parsear la variacion '{variation_text}': {e}")
    
    def get_quote(self) -> DollarQuote:
        soup = self._fetch_page()
        
        price = self._parse_price(soup)
        variation_pct, variation_value = self._parse_variation(soup)
        
        return DollarQuote(
            price=price,
            variation_pct=variation_pct,
            variation_value=variation_value,
            timestamp=datetime.now().isoformat(),
            download_time=self._download_time,
        )
    
    def get_price(self) -> float:
        return self.get_quote().price
    
    def get_variation(self) -> float:
        return self.get_quote().variation_pct
    
    @property
    def last_download_time(self) -> Optional[str]:
        return self._download_time


def get_dollar_price() -> float:
    return DollarScraper().get_price()


def get_dollar_quote() -> DollarQuote:
    return DollarScraper().get_quote()


def get_dollar_dict() -> dict:
    return DollarScraper().get_quote().to_dict()


if __name__ == "__main__":
    print("Obteniendo cotizacion del dolar...")
    print("=" * 50)
    
    try:
        scraper = DollarScraper()
        quote = scraper.get_quote()
        
        print(f"Precio:        {quote.formatted_price}")
        print(f"Variacion:     {quote.formatted_variation}")
        print(f"Hora descarga: {quote.download_time}")
        print(f"Timestamp:     {quote.timestamp}")
        
    except DollarScraperError as e:
        print(f"Error: {e}")

Obteniendo cotizacion del dolar...
Precio:        $3,699.50 COP
Variacion:     -0.54%
Hora descarga: 2025-12-29 14:13:06
Timestamp:     2025-12-29T14:13:08.287961


In [68]:
pd.DataFrame(quote.to_dict(), index=[0])

,price,variation_pct,variation_value,timestamp,download_time,source,is_positive
0,3699.5041,-0.54,0.0,2025-12-29T14:13:08.287961,2025-12-29 14:13:06,Google Finance,False


#### Euro


In [76]:

@dataclass
class EuroQuote:
    price: float
    variation_pct: float
    download_time: str
    source: str = "Google Finance"
    
    @property
    def is_positive(self) -> bool:
        return self.variation_pct >= 0
    
    @property
    def formatted_price(self) -> str:
        return f"${self.price:,.2f} COP"
    
    @property
    def formatted_variation(self) -> str:
        sign = "+" if self.is_positive else ""
        return f"{sign}{self.variation_pct:.2f}%"
    
    def to_dict(self) -> dict:
        return {
            "price": self.price,
            "variation_pct": self.variation_pct,
            "download_time": self.download_time,
            "source": self.source,
            "is_positive": self.is_positive,
        }


class EuroScraperError(Exception):
    pass


class EuroScraper:
    
    URL = "https://www.google.com/finance/quote/EUR-COP"
    
    HEADERS = {
        "User-Agent": (
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0.0.0 Safari/537.36"
        )
    }
    
    def __init__(self, timeout: int = 10):
        self.timeout = timeout
        self._download_time: Optional[str] = None
    
    def _fetch_page(self) -> BeautifulSoup:
        try:
            self._download_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            
            response = requests.get(
                self.URL,
                headers=self.HEADERS,
                timeout=self.timeout
            )
            response.raise_for_status()
            return BeautifulSoup(response.content, "html.parser")
        
        except requests.RequestException as e:
            raise DollarScraperError(f"Error al obtener la pagina: {e}")
    
    def _parse_price(self, soup: BeautifulSoup) -> float:
        element = soup.find("div", class_="YMlKec fxKbKc")
        
        if element is None:
            raise DollarScraperError("No se encontro el elemento del precio")
        
        price_text = element.text.strip()
        price_clean = price_text.replace("$", "").replace(",", "")
        
        try:
            return float(price_clean)
        except ValueError:
            raise DollarScraperError(f"No se pudo convertir el precio: {price_text}")
    
    def _parse_variation(self, soup: BeautifulSoup) -> tuple[float, float]:
        element = soup.find("div", class_="JwB6zf")
        
        if element is None:
            raise DollarScraperError("No se encontro el elemento de variacion")
        
        variation_text = element.text.strip()
        
        try:
            if variation_text.endswith("%") and " " not in variation_text:
                pct_text = variation_text.replace("+", "").replace("%", "")
                variation_pct = float(pct_text)
                variation_value = 0.0  
                return variation_pct, variation_value
            
            
            parts = variation_text.split(" ")
            
            value_text = parts[0].replace("+", "").replace(",", "")
            variation_value = float(value_text)
            
            pct_text = parts[1].replace("(", "").replace(")", "").replace("%", "")
            variation_pct = float(pct_text)
            
            if variation_text.startswith("-"):
                variation_value = -abs(variation_value)
                variation_pct = -abs(variation_pct)
            
            return variation_pct, variation_value
            
        except (IndexError, ValueError) as e:
            raise DollarScraperError(f"No se pudo parsear la variacion '{variation_text}': {e}")
    
    def get_quote(self) -> EuroQuote:
        soup = self._fetch_page()
        
        price = self._parse_price(soup)
        variation_pct, variation_value = self._parse_variation(soup)
        
        return EuroQuote(
            price=price,
            variation_pct=variation_pct,
            download_time=self._download_time,
        )
    
    def get_price(self) -> float:
        return self.get_quote().price
    
    def get_variation(self) -> float:
        return self.get_quote().variation_pct
    
    @property
    def last_download_time(self) -> Optional[str]:
        return self._download_time


def get_euro_price() -> float:
    return EuroScraper().get_price()


def get_euro_quote() -> EuroQuote:
    return EuroScraper().get_quote()


def get_euro_dict() -> dict:
    return EuroScraper().get_quote().to_dict()

if __name__ == "__main__":
    print("Obteniendo cotizacion del Euro...")
    print("=" * 50)
    
    try:
        scraper = EuroScraper()
        quote = scraper.get_quote()
        
        print(f"Precio:        {quote.formatted_price}")
        print(f"Variacion:     {quote.formatted_variation}")
        print(f"Hora descarga: {quote.download_time}")
        
    except DollarScraperError as e:
        print(f"Error: {e}")
    
    


Obteniendo cotizacion del Euro...
Precio:        $4,350.60 COP
Variacion:     -1.68%
Hora descarga: 2025-12-29 14:27:16
